### Linear regression ###

#### Notes 

To-Do  
  
1. Hyperparamter tuning
    1. Do the scan in the defined boundary
    2. Gather the best model
    3. Read about GPU usage in Talos
2. Saving the best model to ONNX format
    1. Use deploy to save as JSON
    2. save method in Keras
    3. ONNX options
        + Use onnxmltool
    4. Is there an option to retrieve the best model from the  
    Talos scan_object without using Deploy.
    5. Problem with Deploy -- you have to save it locally and then read it back
3. Research on the best models
4. LaTeX equations
5. Create decorator
    + Returns ONNX model
    + Writes keras and scikit params to master sheet

#### Scikit ####

In [4]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler

diabetes = datasets.load_diabetes()

# Use only one feature
diabetes_X = diabetes.data[:, np.newaxis, 2]
sc = StandardScaler()
diabetes.data = sc.fit_transform(diabetes.data)

x = diabetes_X
y = diabetes.target


# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes.target[:-20]
diabetes_y_test = diabetes.target[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# Make predictions using the testing set
diabetes_y_pred = regr.predict(diabetes_X_test)

score = regr.score(diabetes_X_test,diabetes_y_test)
mean_squared_error(diabetes_y_test, diabetes_y_pred)

2548.0723987259703

In [27]:
diabetes.data

array([[ 0.80050009,  1.06548848,  1.29708846, ..., -0.05449919,
         0.41855058, -0.37098854],
       [-0.03956713, -0.93853666, -1.08218016, ..., -0.83030083,
        -1.43655059, -1.93847913],
       [ 1.79330681,  1.06548848,  0.93453324, ..., -0.05449919,
         0.06020733, -0.54515416],
       ...,
       [ 0.87686984,  1.06548848, -0.33441002, ..., -0.23293356,
        -0.98558469,  0.32567395],
       [-0.9560041 , -0.93853666,  0.82123474, ...,  0.55838411,
         0.93615545, -0.54515416],
       [-0.9560041 , -0.93853666, -1.53537419, ..., -0.83030083,
        -0.08871747,  0.06442552]])

#### Keras ###

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD, Nadam
from keras.losses import mse
from keras.activations import linear
import talos as ta

# np.random.seed(7)

# The parameter 'p' should be dynamic and a part of the mapping

p = {'lr': (2, 10, 30),
     'first_neuron': [1],
     'batch_size': [1, 2, 3, 4],
     'epochs': [10,20,40],
     'weight_regulizer': [None],
     'emb_output_dims': [None],
     'optimizer': ['SGD', 'nadam'],
     'losses': [mse],
     'activation': [linear]
    }

def keras_model(x_train, y_train, x_val, y_val, params):
    model = Sequential()
    model.add(Dense(params['first_neuron'],
                    input_dim=x_train.shape[1],
                    activation=params['activation']))

    model.compile(optimizer=params['optimizer'],
                  loss=params['losses'],
                  metrics=['mse'])

    out = model.fit(diabetes_X_train, diabetes_y_train,
                    batch_size=params['batch_size'],
                    epochs=params['epochs'],
                    verbose=0,
                    validation_data=[x_val, y_val])
    
    return out, model
    

    
h = ta.Scan(x, y,
            params=p,
            dataset_name='first_linear_regression',
            experiment_no='a',
            model=keras_model,
            grid_downsample=0.5)
    
    


100%|██████████| 96/96 [16:11<00:00,  9.44s/it]


Scan Finished!


In [9]:
# Testing deploy from Talos 
from talos import Deploy

Deploy(h, 'linear_regression_firstDataset',metric='val_loss')

Deploy package linear_regression_firstDataset have been saved.


In [41]:
# loading the best model

import os,json,zipfile,shutil
from keras.models import model_from_json
archive = zipfile.ZipFile('linear_regression_firstDataset.zip', 'r')
model_file = archive.open('linear_regression_firstDataset_model.json')
weight_file = archive.open('linear_regression_firstDataset_model.h5')

with zipfile.ZipFile('linear_regression_firstDataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./linear_regression_firstDataset_unzip')

# json_file = open('model.json', 'r')
loaded_model_json = model_file.read()

# json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./linear_regression_firstDataset/linear_regression_firstDataset_model.h5")


shutil.rmtree('./linear_regression_firstDataset_unzip')
print("Loaded model from disk")

Loaded model from disk


In [42]:
type(loaded_model)

keras.engine.sequential.Sequential

In [44]:
# Testing ONNX ml tool
import onnxmltools
onnx_model = onnxmltools.convert_keras(loaded_model)

The maximum opset needed by this model is only 7.


In [45]:
onnx_model

ir_version: 3
producer_name: "OnnxMLTools"
producer_version: "1.3.1"
domain: "onnxml"
model_version: 0
doc_string: ""
graph {
  node {
    input: "dense_1_input_14_0"
    input: "W"
    output: "transformed_tensor"
    name: "_class__keras_layers_core_Dense__"
    op_type: "MatMul"
    domain: ""
  }
  node {
    input: "transformed_tensor"
    input: "B"
    output: "biased_tensor_name"
    name: "Add"
    op_type: "Add"
    domain: ""
  }
  node {
    input: "biased_tensor_name"
    output: "dense_1_14_BiasAdd_01"
    name: "Identity"
    op_type: "Identity"
    domain: ""
  }
  name: "e704ad28cdf745b2bf06ded1ab6e743d"
  initializer {
    dims: 1
    dims: 1
    data_type: FLOAT
    float_data: 162.32818603515625
    name: "W"
  }
  initializer {
    dims: 1
    data_type: FLOAT
    float_data: 153.9892578125
    name: "B"
  }
  input {
    name: "dense_1_input_14_0"
    type {
      tensor_type {
        elem_type: FLOAT
        shape {
          dim {
            dim_value: 1
     

In [42]:
keras_score = model.evaluate(diabetes_X_test, diabetes_y_pred)
print(model.metrics_names[0],':', keras_score[0])

20/20 [==============================] - 0s 10ms/step
loss : 1956.4066162109375


#### Notes ####

1) MSE or accuracy? What is the right metric in this case?  
2) Try altering
    - learning rate
    - optimizer or loss function
    - 
3) **Normailzation of data**  
4) Check if anything is wrong with model.evaluate  
5) Size of loss too large?  
6) Why is the epoch mse value different from the final evaluated value?  
7) How to validate if this dnn model is the right representation of linear regression?     
The math behind the Keras code  
8)  

***

### SVM ###

#### Scikit ####

In [9]:
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv("../data/iris.csv" , delimiter=",", header=None, index_col=False)
class_name,index = np.unique(data.iloc[:,-1],return_inverse=True)
data.iloc[:,-1] = index
data = data.loc[data[4] != 2]
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.60, random_state=0)

model = svm.SVC(kernel='linear', C=1, gamma=1)

model.fit(x_train, y_train)
score = model.score(x_test, y_test)

prediction= model.predict(x_test)

print(score)

1.0


In [11]:
prediction.shape

(60,)

#### Keras ####

In [41]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.regularizers import l2

np.random.seed(7)

model = Sequential()
model.add(Dense(1,input_dim=4,activation='linear', W_regularizer=l2(0.01)))

model.compile(loss='categorical_hinge', optimizer='adadelta', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=120, batch_size=10)

C:\Users\shakk\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, input_dim=4, activation="linear", kernel_regularizer=<keras.reg...)`
  


Epoch 1/120
40/40 [==============================] - 0s 10ms/step - loss: 1.7212 - acc: 0.1000
Epoch 2/120
40/40 [==============================] - 0s 391us/step - loss: 1.6915 - acc: 0.2000
Epoch 3/120
40/40 [==============================] - 0s 391us/step - loss: 1.6610 - acc: 0.3000
Epoch 4/120
40/40 [==============================] - 0s 391us/step - loss: 1.6308 - acc: 0.3750
Epoch 5/120
40/40 [==============================] - 0s 391us/step - loss: 1.6032 - acc: 0.5000
Epoch 6/120
40/40 [==============================] - 0s 391us/step - loss: 1.5788 - acc: 0.5250
Epoch 7/120
40/40 [==============================] - 0s 390us/step - loss: 1.5583 - acc: 0.5250
Epoch 8/120
40/40 [==============================] - 0s 391us/step - loss: 1.5412 - acc: 0.5500
Epoch 9/120
40/40 [==============================] - 0s 390us/step - loss: 1.5288 - acc: 0.5500
Epoch 10/120
40/40 [==============================] - 0s 519us/step - loss: 1.5187 - acc: 0.5500
Epoch 11/120
40/40 [====================

In [42]:
keras_score = model.evaluate(x_test, prediction)
print(model.metrics_names[1],':', keras_score[1])

60/60 [==============================] - 0s 3ms/step
acc : 1.0


---

### Naive Bayes

#### Scikit

In [45]:
from sklearn.naive_bayes import GaussianNB 

model = GaussianNB()

model.fit(x_train,y_train)

predicted= model.predict(x_test)

score = model.score(x_test, y_test)
print(score)

1.0


#### Keras